In [1]:
from azureml.core import Workspace, Experiment, Environment, Dataset, ScriptRunConfig, Run
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import RandomParameterSampling, uniform, choice
from azureml.core.compute import ComputeTarget
from azureml.train.sklearn import SKLearn 
%reload_ext sql
import pandas as pd

In [2]:
ws = Workspace.from_config()

In [3]:
cpu_cluster_name = "cpuclusternew"

from azureml.core.compute import AmlCompute

compute = AmlCompute(ws, cpu_cluster_name)

In [4]:
param_sampling = RandomParameterSampling({
    '--stop_loss': choice([1,2,3]),
    '--take_profit': choice([1,2,3,4]),
    '--n_estimators': choice([50,100,120,150,200,250]),
    '--class_weight_0': choice([1, 5, 6, 10, 15,20]),
    '--class_weight_1': choice([1, 5, 6, 10, 15,20]),
    '--max_features': choice('sqrt','log2'),
    '--random_state': choice([0,8,26,42]),
    '--n_components': choice ([10, 15, 20, 25, 30]),
    '--threshold': choice([0.5,0.55,0.56,0.57,0.6]),
    '--fdr_level': choice([0.05,0.06,0.08,0.10]),
    '--highcorr':choice([0.8,0.9])
})

In [5]:
arguments = ['--dataset_name', 'EURUSD_D1_2010to101124']

forex_env = Environment.get(workspace=ws,name='forex_env')

# Create a ScriptRunConfig
src = ScriptRunConfig(source_directory='./scripts',
                      script='train_script_buy.py',
                      arguments=arguments,
                      environment=forex_env,
                      compute_target=compute)

In [6]:
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=param_sampling,
                                     primary_metric_name='overall_precision',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=500,
                                     max_concurrent_runs=50)

In [7]:
experiment = Experiment(workspace=ws, name='EURUSD2010till101124Buy')
hyperdrive_run = experiment.submit(hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_8491e832-51c8-4c1c-a933-ff62c40fd992
Web View: https://ml.azure.com/runs/HD_8491e832-51c8-4c1c-a933-ff62c40fd992?wsid=/subscriptions/a960077c-b037-4e6a-9491-8d9d841776de/resourcegroups/forex_resource/workspaces/forexbuild&tid=34406be1-a157-487e-a6f0-25db6b86ea91

Streaming azureml-logs/hyperdrive.txt

[2025-01-06T22:48:09.3701068Z][GENERATOR][DEBUG]Sampled 50 jobs from search space 
[2025-01-06T22:48:09.8837902Z][SCHEDULER][INFO]Scheduling job, id='HD_8491e832-51c8-4c1c-a933-ff62c40fd992_2' 
[2025-01-06T22:48:09.7994921Z][SCHEDULER][INFO]Scheduling job, id='HD_8491e832-51c8-4c1c-a933-ff62c40fd992_0' 
[2025-01-06T22:48:09.9982321Z][SCHEDULER][INFO]Scheduling job, id='HD_8491e832-51c8-4c1c-a933-ff62c40fd992_3' 
[2025-01-06T22:48:09.8848531Z][SCHEDULER][INFO]Scheduling job, id='HD_8491e832-51c8-4c1c-a933-ff62c40fd992_1' 
[2025-01-06T22:48:09.9997304Z][SCHEDULER][INFO]Scheduling job, id='HD_8491e832-51c8-4c1c-a933-ff62c40fd992_5' 
[2025-01-06T22:48:10.0014309Z][SCHEDULER][INFO]Sc

In [ ]:
import pandas as pd

# Get all child runs
child_runs = hyperdrive_run.get_children()

# Initialize a list to store aggregated metrics and hyperparameters
aggregated_info = []

# Iterate over child runs and fetch their metrics and hyperparameters
for child_run in child_runs:
    child_metrics = child_run.get_metrics()
    child_hyperparameters = child_run.get_details()['runDefinition']['arguments']

    # Convert arguments list to a dictionary of hyperparameters
    hyperparameters_dict = {child_hyperparameters[i]: child_hyperparameters[i+1] for i in range(0, len(child_hyperparameters), 2)}
    
    # Fetch metrics that were originally obtained
    precision_splits = [child_metrics.get(f'precision_split_{i}', None) for i in range(1, 6)]
    overall_precision = child_metrics.get('overall_precision', None)
    no_of_trades = child_metrics.get('no_of_trades', None)
    adj_takeprofit = child_metrics.get('adj_takeprofit', None)
    WIN_LOSS_Diff_4 = child_metrics.get('WIN/LOSS-Diff_4', None)
    WIN_LOSS_Diff_5 = child_metrics.get('WIN/LOSS-Diff_5', None)
    BreakEvenRatio = child_metrics.get('BreakEvenRatio', None)

    # Fetch additional metrics
    false_positives = child_metrics.get('false_positives', None)
    true_positives = child_metrics.get('true_positives', None)
    early_stopping_triggered = child_metrics.get('early_stopping_triggered', None)
    break_even_ratio = child_metrics.get('break_even_ratio', None)
    precision = child_metrics.get('precision', None)  # If early stopping triggered, might be logged
    overall_trades = child_metrics.get('overall_trades', None)

    run_details = {
        'run_id': child_run.id,
        'adj_takeprofit': adj_takeprofit,
        'WIN/LOSS-Diff_4': WIN_LOSS_Diff_4,
        'WIN/LOSS-Diff_5': WIN_LOSS_Diff_5,
        'overall_precision': overall_precision,
        'BreakEvenRatio': BreakEvenRatio,
        'no_of_trades': no_of_trades,
        'false_positives': false_positives,
        'true_positives': true_positives,
        'early_stopping_triggered': early_stopping_triggered,
        'precision': precision,
        'overall_trades': overall_trades,
        **hyperparameters_dict,
        **{f'precision_split_{i}': precision_splits[i-1] for i in range(1, 6)},
    }

    aggregated_info.append(run_details)

# Convert the aggregated information to a pandas DataFrame
df = pd.DataFrame(aggregated_info)

# Sort the DataFrame by a chosen metric, for instance 'WIN/LOSS-Diff_4' in descending order
df = df.sort_values(by='WIN/LOSS-Diff_4', ascending=False)

# Display the DataFrame
print(df)

# Save the DataFrame to a CSV file
csv_file_path = 'EURUSD2010till101124BUY1.csv'  # Specify your desired path
df.to_csv(csv_file_path, index=False)

In [ ]:
from azureml.core import Experiment, Run

# Replace with the name of the experiment in which the HyperDrive run was created
experiment_name = "GBPUSD2010till101124SELL"

# Get (or create) the experiment object
experiment = Experiment(workspace=ws, name=experiment_name)

# Retrieve the HyperDrive run by run ID
hyperdrive_run = Run(experiment=experiment, run_id="HD_576c925a-dbae-41a9-9e73-9db367e6c963 ")

import pandas as pd

# Get all child runs
child_runs = hyperdrive_run.get_children()

aggregated_info = []

for child_run in child_runs:
    child_metrics = child_run.get_metrics()
    child_hyperparameters = child_run.get_details()['runDefinition']['arguments']

    # Convert arguments list to a dict of hyperparameters
    hyperparameters_dict = {
        child_hyperparameters[i]: child_hyperparameters[i+1] 
        for i in range(0, len(child_hyperparameters), 2)
    }

    precision_splits = [child_metrics.get(f'precision_split_{i}', None) for i in range(1, 6)]
    overall_precision = child_metrics.get('overall_precision', None)
    no_of_trades = child_metrics.get('no_of_trades', None)
    adj_takeprofit = child_metrics.get('adj_takeprofit', None)
    WIN_LOSS_Diff_4 = child_metrics.get('WIN/LOSS-Diff_4', None)
    WIN_LOSS_Diff_5 = child_metrics.get('WIN/LOSS-Diff_5', None)
    BreakEvenRatio = child_metrics.get('BreakEvenRatio', None)

    # Fetch additional metrics that might be logged
    false_positives = child_metrics.get('false_positives', None)
    true_positives = child_metrics.get('true_positives', None)
    early_stopping_triggered = child_metrics.get('early_stopping_triggered', None)
    break_even_ratio = child_metrics.get('break_even_ratio', None)
    precision = child_metrics.get('precision', None)
    overall_trades = child_metrics.get('overall_trades', None)

    run_details = {
        'run_id': child_run.id,
        'adj_takeprofit': adj_takeprofit,
        'WIN/LOSS-Diff_4': WIN_LOSS_Diff_4,
        'WIN/LOSS-Diff_5': WIN_LOSS_Diff_5,
        'overall_precision': overall_precision,
        'BreakEvenRatio': BreakEvenRatio,
        'no_of_trades': no_of_trades,
        'false_positives': false_positives,
        'true_positives': true_positives,
        'early_stopping_triggered': early_stopping_triggered,
        'precision': precision,
        'overall_trades': overall_trades,
        **hyperparameters_dict,
        **{f'precision_split_{i}': precision_splits[i-1] for i in range(1, 6)},
    }

    aggregated_info.append(run_details)

df = pd.DataFrame(aggregated_info)

# Sort the DataFrame by a chosen metric
df = df.sort_values(by='WIN/LOSS-Diff_4', ascending=False)

print(df)

# Save to CSV
csv_file_path = 'GBPUSD2010till101124SELL.csv'
df.to_csv(csv_file_path, index=False)


In [ ]:
df

In [ ]:
df_dropped = df.dropna()

In [ ]:
df_dropped

In [ ]:
droppedsell = df_dropped[df_dropped['precision_split_5'] > df_dropped['BreakEvenRatio']] 

In [ ]:
droppedsell

In [ ]:
csv_file_path = 'dropped_ICMT5USDCAD_2010_3112_D1_Sell.csv'
droppedsell.to_csv(csv_file_path,index=False)